# Generation of observation from inhomogenous Poisson process

## Algorithm by Lewis and Shedler, 1979

In [ ]:
import numpy as np
from Generator import LewisShedler

In [ ]:
T = np.pi
n_size = 1000

In [ ]:
def lam(t):
    return np.where(t < 1, n_size, n_size*(np.sin(t**2*np.pi)+1)/3)

In [ ]:
generator = LewisShedler(lam, T, lambda_hat=n_size)

In [ ]:
%%timeit 
t = generator.generate()

In [ ]:
t = generator.generate()
print('Number of simulated points: {}'.format(len(t)))

In [ ]:
generator.visualize(save=False)

# Approximation of integral operator

In [ ]:
import numpy as np
import dask.array as da
from dask.diagnostics import ProgressBar
from Operator import Operator

In [ ]:
def kernel(x, y):
    return np.repeat(1, len(x))

In [ ]:
operator = Operator(kernel, 0, 1, 500, "rectangle")

In [ ]:
oper = operator.approximate(compute=False)

In [ ]:
with ProgressBar():
    oper = operator.K.compute()
oper.shape

# Estimation of intensity function using Landweber iteration

In [ ]:
sample_size = 100

In [ ]:
def kernel(x, y):
    return np.where(x < y, 1, 1)

In [ ]:
def lam(t):
    return np.where(t<1, sample_size, sample_size)

In [ ]:
from Generator import LewisShedler
generator = LewisShedler(lam, 1)
obs = generator.generate()
generator.visualize()

In [ ]:
from Estimators import Landweber

In [ ]:
landwerber = Landweber(kernel, 0, 1, 500, obs, sample_size, 'rectangle', relaxation=0.05, max_iter=200)

In [ ]:
landwerber.observations = generator.generate()
landwerber.refresh()
landwerber.observations

In [ ]:
landwerber.q_estimator.compute()

In [ ]:
landwerber.delta.compute()

In [ ]:
landwerber.estimate(compute=True)

In [ ]:
landwerber.solution

In [ ]:
landwerber.L2norm(landwerber.solution, np.repeat([1], landwerber.solution.shape[0]), sqrt=True).compute()